Criteria 1.3 name check.


*   wrong but caller mai dai arai --> not pass
*   wrong then correct --> pass

*  have คุณ infront of the correct name --> pass
*  have คุณ infront of the name but not correct name --> wrong







In [31]:
text = '''**Speaker 1 (Contact Center Agent):**
"1330 สวัสดีค่ะ มิน รับสาย ยินดีให้บริการค่ะ"

**Speaker 2 (Citizen):**
"สวัสดีครับ ขอสอบถามเรื่องการลงทะเบียนสิทธิประกันสุขภาพแห่งชาติครับ"

**Speaker 1:**
"ได้ค่ะ ขอทราบชื่อ-นามสกุลและเลขบัตรประจำตัวประชาชนของท่านก่อนได้ไหมคะ?"

**Speaker 2:**
"ชื่อนายเอกลักษณ์ ชัยชนะ ครับ เลขบัตรประชาชน 1-2345-67890-12-3 ครับ"

**Speaker 1:**
"ขอบคุณค่ะ คุณเฟรม กรุณาบอกเล่าเรื่องราวให้ฟังหน่อยได้ไหมคะ ว่าต้องการความช่วยเหลือเรื่องอะไร?"

**Speaker 2:**
"ได้ครับ อยากทราบว่าการลงทะเบียนสิทธิประกันสุขภาพแห่งชาติสามารถทำได้ที่ไหนครับ?"

**Speaker 1:**
"ค่ะ สำหรับการลงทะเบียนสิทธิประกันสุขภาพแห่งชาติ ท่านสามารถทำได้ที่โรงพยาบาลหรือคลินิกใกล้บ้านที่เป็นหน่วยบริการประจำของท่าน หรือท่านสามารถติดต่อสำนักงานหลักประกันสุขภาพแห่งชาติ (สปสช.) สาขาที่ท่านอยู่ได้ค่ะ"

**Speaker 2:**
"ถ้าผมอยู่ที่จังหวัดเชียงใหม่ ต้องติดต่อที่ไหนครับ?"

**Speaker 1:**
"ถ้าคุณเอกลักษณ์อยู่ที่จังหวัดเชียงใหม่ ท่านสามารถติดต่อสำนักงานหลักประกันสุขภาพแห่งชาติ (สปสช.) ภาค 1 จังหวัดเชียงใหม่ ได้ค่ะ หรือท่านสามารถติดต่อที่โรงพยาบาลหรือคลินิกใกล้บ้านของท่านเพื่อลงทะเบียนได้เช่นกัน"

**Speaker 2:**
"แล้วถ้าผมมีปัญหาในการลงทะเบียนล่ะครับ? สามารถติดต่อใครได้บ้าง?"

**Speaker 1:**
"หากท่านมีปัญหาในการลงทะเบียน ท่านสามารถติดต่อเจ้าหน้าที่ที่โรงพยาบาลหรือคลินิกที่ท่านลงทะเบียน หรือติดต่อสำนักงานหลักประกันสุขภาพแห่งชาติ (สปสช.) ภาค 1 จังหวัดเชียงใหม่ ได้ค่ะ"

**Speaker 2:**
"ขอบคุณมากครับ"

**Speaker 1:**
"ยินดีค่ะ ขอให้ท่านมีสุขภาพแข็งแรง และถ้ามีคำถามเพิ่มเติม ติดต่อเราได้ตลอด 24 ชั่วโมง ครับ สวัสดีครับ"'''


In [32]:
import re

def check_agent_name_usage_clean(text):
    # Parse conversation with a regex pattern
    turns = []
    pattern = r'\*\*Speaker (\d+)(?:\s*\([^)]*\))?:\*\*\s*\n"([^"]+)"'
    matches = re.findall(pattern, text, re.DOTALL)

    if not matches:
        return "Error: Could not parse conversation."

    for speaker_num, content in matches:
        role = "agent" if speaker_num == "1" else "customer"
        turns.append({
            "role": role,
            "content": content.strip(),
            "speaker_num": speaker_num
        })

    # Find where customer provides their name
    customer_name = None
    for turn in turns:
        if turn["role"] == "customer" and "ชื่อ" in turn["content"]:
            parts = turn["content"].split("ชื่อ")
            if len(parts) > 1:
                name_chunk = parts[1].split("ครับ")[0].strip()
                for title in ["นาย", "นาง", "นางสาว"]:
                    if name_chunk.startswith(title):
                        name_chunk = name_chunk.replace(title, "", 1).strip()
                        break
                name_parts = name_chunk.split()
                if len(name_parts) >= 2:
                    customer_name = {
                        "first_name": name_parts[0],
                        "last_name": name_parts[1]
                    }
                    break

    if not customer_name:
        return "Could not extract customer name from conversation."

    # Check if agent uses "คุณ + name"
    agent_name_usage = []
    for i, turn in enumerate(turns):
        if turn["role"] == "agent":
            proper_address = f"คุณ{customer_name['first_name']}"
            if proper_address in turn["content"]:
                agent_name_usage.append({
                    "turn_index": i,
                    "content": turn["content"]
                })

    if agent_name_usage:
        return 1  # Return 1 for PASS
    else:
        for turn in turns:
            if turn["role"] == "agent" and customer_name["first_name"] in turn["content"]:
                break
        return 0  # Return 0 for NOT PASS

In [34]:
label = check_agent_name_usage_clean(text)
print(label)

1
